In [1]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import random
import time

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import KFold,StratifiedKFold
from skimage.transform import resize
import sklearn
import os
import gc
import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, SequentialSampler
from torch.utils.tensorboard import SummaryWriter
from timm.models import resnet34d, resnet34, resnext50d_32x4d, densenet121, resnest50d
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
torch.__version__

'1.7.1'

In [3]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    
GLOBAL_SEED = 12415
setup_seed(GLOBAL_SEED)

In [4]:
data_path = '/dev/shm/data'
feat_path = '/root/s/RFCX/features'
res_path = '/root/s/RFCX/res'
model_path = '/dev/shm/model_save'
tensorboard_path = '/root/s/RFCX/tensorboard'
if not os.path.exists(model_path):
    os.makedirs(model_path)
if not os.path.exists(res_path):
    os.makedirs(res_path)
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)

In [5]:
data_tp_df=pd.read_csv(os.path.join(data_path, 'train_tp.csv'))
data_fp_df=pd.read_csv(os.path.join(data_path, 'train_fp.csv'))

## Some Global Parameter

In [6]:
class Config:
    num_class = 24
    n_fft = 2048
    hop_length = 512
    n_mels = 256
    sr = 32000
    segment_length = 10 * sr
    fmin = 80
    fmax = 16000
    
    resize = False
    img_shape = (256, 400)
    
    wav_augment = True
    spec_augment = True
    mixup_proba = 0.5
    mixup_alpha = 5
    
    model = resnet34d
    model_name = 'resnet34d_augment_RandomCrop_mixup_attention'

## Prepare Dataset and Dataloader

In [7]:
"https://www.kaggle.com/gopidurgaprasad/audio-augmentation-albumentations/"

import matplotlib.pyplot as plt
import IPython.display as ipd
import albumentations
from albumentations.core.transforms_interface import DualTransform, BasicTransform
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, PolarityInversion, Gain, AddGaussianSNR, TimeMask, FrequencyMask


class AudioTransform(BasicTransform):
    """Transform for Audio task"""

    @property
    def targets(self):
        return {"data": self.apply}
    
    def update_params(self, params, **kwargs):
        if hasattr(self, "interpolation"):
            params["interpolation"] = self.interpolation
        if hasattr(self, "fill_value"):
            params["fill_value"] = self.fill_value
        return params
    
      
class MelSpectrogram(AudioTransform):
    """Shifting time axis"""
    def __init__(self, parameters, always_apply=False, p=0.5):
        super(MelSpectrogram, self).__init__(always_apply, p)

        self.parameters = parameters
    
    def apply(self, data, **params):
        sound, sr = data
        melspec = librosa.feature.melspectrogram(sound, sr=sr, **self.parameters)
        melspec = librosa.power_to_db(melspec)
        melspec = melspec.astype(np.float32)
        return melspec, sr

    
class SpecAugment(AudioTransform):
    """Shifting time axis"""
    def __init__(self, num_mask=2, freq_masking=0.15, time_masking=0.20, always_apply=False, p=0.5):
        super(SpecAugment, self).__init__(always_apply, p)

        self.num_mask = num_mask
        self.freq_masking = freq_masking
        self.time_masking = time_masking
    
    def apply(self, data, **params):
        melspec, sr = data
        spec_aug = self.spec_augment(melspec, 
                                     self.num_mask,
                                     self.freq_masking,
                                     self.time_masking,
                                     melspec.min())
        return spec_aug, sr
    
    # Source: https://www.kaggle.com/davids1992/specaugment-quick-implementation
    def spec_augment(self, 
                    spec: np.ndarray,
                    num_mask=2,
                    freq_masking=0.15,
                    time_masking=0.20,
                    value=0):
        spec = spec.copy()
        num_mask = random.randint(1, num_mask)
        for i in range(num_mask):
            all_freqs_num, all_frames_num  = spec.shape
            freq_percentage = random.uniform(0.0, freq_masking)
            num_freqs_to_mask = int(freq_percentage * all_freqs_num)
            f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
            f0 = int(f0)
            spec[f0:f0 + num_freqs_to_mask, :] = value

            time_percentage = random.uniform(0.0, time_masking)

            num_frames_to_mask = int(time_percentage * all_frames_num)
            t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
            t0 = int(t0)
            spec[:, t0:t0 + num_frames_to_mask] = value

        return spec

    
class SpectToImage(AudioTransform):

    def __init__(self, always_apply=False, p=0.5):
        super(SpectToImage, self).__init__(always_apply, p)
        
        
    def mono_to_color(self, X: np.ndarray,
                      mean=None,
                      std=None,
                      norm_max=None,
                      norm_min=None,
                      eps=1e-6):
        """
        Code from https://www.kaggle.com/daisukelab/creating-fat2019-preprocessed-data
        """


        # Standardize
        mean = mean or X.mean()
        X = X - mean
        std = std or X.std()
        Xstd = X / (std + eps)
        _min, _max = Xstd.min(), Xstd.max()
        norm_max = norm_max or _max
        norm_min = norm_min or _min
        if (_max - _min) > eps:
            # Normalize to [0, 255]
            V = Xstd
            V[V < norm_min] = norm_min
            V[V > norm_max] = norm_max
            V = (V - norm_min) / (norm_max - norm_min)
        else:
            # Just zero
            V = np.zeros_like(Xstd, dtype=np.float32)
        return V
    
    
    def apply(self, data, **params):
        melspec, sr = data
        image = self.mono_to_color(melspec)
        if Config.resize:
            image = resize(image, Config.img_shape)
        image = np.stack([image, image, image], axis=-1)
#         delta = librosa.feature.delta(image)
#         accelerate = librosa.feature.delta(image, order=2)
#         image = np.stack([image, delta, accelerate], axis=-1)
#         image = image.astype(np.float32) / 100.0
        # (n_mels, time_step, 3) --> (3, time_step, n_mels)
        return image.transpose(2, 1, 0)

    




sound_augment = Compose([
    PolarityInversion(p=0.2),
    Gain(min_gain_in_db=-15, max_gain_in_db=15, p=0.3),
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.1),
    AddGaussianSNR(max_SNR=0.5, p=0.2),
    FrequencyMask(min_frequency_band=0.01,  max_frequency_band=0.25, p=0.2),
    TimeMask(min_band_part=0.01, max_band_part=0.25, p=0.2)
#     TimeStretch(min_rate=0.8, max_rate=1.25, p=0.2)
#     Shift(min_fraction=-0.2, max_fraction=0.2, p=0.2)
])


melspectrogram_parameters = {
        "n_mels": Config.n_mels,
        'n_fft': Config.n_fft, 
        'hop_length': Config.hop_length,
        'fmin': Config.fmin, 
        'fmax': Config.fmax 
    }

spec_augment = albumentations.Compose([
    MelSpectrogram(parameters=melspectrogram_parameters, always_apply=True),
    SpecAugment(p=0.2),
    SpectToImage(always_apply=True)
])

to_image = albumentations.Compose([
    MelSpectrogram(parameters=melspectrogram_parameters, always_apply=True),
    SpectToImage(always_apply=True)
])


In [8]:
ONE_HOT = np.eye(Config.num_class)
class TrainDataset(Dataset):
    def __init__(self, data_df, is_valid=False):
        self.data_df = data_df
        self.is_valid = is_valid
    
    def __len__(self):
        return len(self.data_df)
    
    def load_audio_clip(self, audio_file_path, t_min, t_max):
        # All sound files are 48000 bitrate, no need to slowly resample
        wav, _ = librosa.load(audio_file_path, sr=Config.sr)

        t_min = float(t_min) * Config.sr
        t_max = float(t_max) * Config.sr

        # Positioning sound slice
        begin = max(t_max - Config.segment_length, 0)
        end = t_min
        random_begin = np.random.randint(begin, end)
        random_end = random_begin + Config.segment_length
        if random_end > len(wav):
            random_end = len(wav)
            random_begin = random_end - Config.segment_length

        slice = wav[int(random_begin):int(random_end)]
        return slice
    
    def __getitem__(self, idx):
        s = self.data_df.iloc[idx]
        audio_file_path = os.path.join(data_path, 'train', s['recording_id']+'.wav')
        wav = self.load_audio_clip(audio_file_path, s['t_min'], s['t_max'])
        if self.is_valid:
            image = to_image(data=(wav, Config.sr))['data']
        else:
            if Config.wav_augment:
                data = sound_augment(samples=wav, sample_rate=Config.sr), Config.sr
            else:
                data = wav, Config.sr
            if Config.spec_augment:
                image = spec_augment(data=data)['data']
            else:
                image = to_image(data=data)['data']
        return torch.tensor(image, dtype=torch.float32), ONE_HOT[s['species_id']]


class TestDataset(Dataset):
    def __init__(self, test_files):
        self.test_files = test_files 
    
    def __len__(self):
        return len(self.test_files)
    
    def __getitem__(self, idx):
        audio_file_path = os.path.join(data_path, 'test', self.test_files[idx])
        wav, _ = librosa.load(audio_file_path, sr=Config.sr)
        segments = len(wav) / Config.segment_length
        segments = int(np.ceil(segments))
        img = []
        for i in range(0, segments):
            # Last segment going from the end
            if (i + 1) * Config.segment_length > len(wav):
                slice = wav[len(wav) - Config.segment_length:len(wav)]
            else:
                slice = wav[i * Config.segment_length:(i + 1) * Config.segment_length]
            img.append(to_image(data=(slice, Config.sr))['data'])
        return torch.tensor(img, dtype=torch.float32)

In [9]:
test_files = sorted(os.listdir(os.path.join(data_path, 'test')))
test_dataset = TestDataset(test_files)
test_dataloader = DataLoader(test_dataset, batch_size=16, sampler=SequentialSampler(test_dataset), shuffle=False, num_workers=4)

In [10]:
batch_size = 32
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=GLOBAL_SEED)
data_folds = []
valid_indexs = []

for idx, (train_index, valid_index) in enumerate(kf.split(X=data_tp_df, y=data_tp_df['species_id'])):
    valid_indexs.append(valid_index)
    train_dataset = TrainDataset(data_tp_df.iloc[train_index], is_valid=False)
    val_dataset = TrainDataset(data_tp_df.iloc[valid_index], is_valid=True)

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    valid_dataloader = DataLoader(val_dataset, batch_size=batch_size, sampler=SequentialSampler(val_dataset), shuffle=False, num_workers=4)
    data_folds.append((train_dataloader, valid_dataloader))

## Build Model and Train

In [11]:
class AttBlock(nn.Module):
    def __init__(self,
                 in_features: int,
                 out_features: int,
                 activation="linear",
                 temperature=1.0):
        super().__init__()

        self.activation = activation
        self.temperature = temperature
        self.att = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)
        self.cla = nn.Conv1d(
            in_channels=in_features,
            out_channels=out_features,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=True)

        self.bn_att = nn.BatchNorm1d(out_features)

    def forward(self, x):
        # x: (n_samples, n_in, n_time)
        energy = torch.tanh(self.att(x))
        norm_att = torch.softmax(energy, dim=-1)
        cla = self.nonlinear_transform(self.cla(x))
        x = torch.sum(norm_att * cla, dim=2)
        return x, norm_att, cla

    def nonlinear_transform(self, x):
        if self.activation == 'linear':
            return x
        elif self.activation == 'sigmoid':
            return torch.sigmoid(x)

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = Config.model(pretrained=True)
        if Config.model_name.startswith('densenet'):
            self.in_features = self.model.classifier.in_features
        else:
            self.in_features = self.model.fc.in_features
        self.fc = nn.Linear(self.in_features, self.in_features, bias=True)
        self.att_block = AttBlock(self.in_features, Config.num_class, activation='sigmoid')
        
        
    def forward(self, X):
        X = self.model.forward_features(X)
        X = torch.mean(X, dim=-1)
        X = X.transpose(1, 2)
        X = F.relu(self.fc(X))
        X = X.transpose(1, 2)
        (clipwise_output, norm_att, segmentwise_output) = self.att_block(X)
        output_dict = {
            'framewise_output': segmentwise_output,
            'clipwise_output': clipwise_output,
        }
        return output_dict

In [12]:
# label-level average
# Assume float preds [BxC], labels [BxC] of 0 or 1
def LWLRAP(preds, labels):
    # Ranks of the predictions
    ranked_classes = torch.argsort(preds, dim=-1, descending=True)
    # i, j corresponds to rank of prediction in row i
    class_ranks = torch.zeros_like(ranked_classes)
    for i in range(ranked_classes.size(0)):
        for j in range(ranked_classes.size(1)):
            class_ranks[i, ranked_classes[i][j]] = j + 1
    # Mask out to only use the ranks of relevant GT labels
    ground_truth_ranks = class_ranks * labels + (1e6) * (1 - labels)
    # All the GT ranks are in front now
    sorted_ground_truth_ranks, _ = torch.sort(ground_truth_ranks, dim=-1, descending=False)
    # Number of GT labels per instance
    num_labels = labels.sum(-1)
    pos_matrix = torch.tensor(np.array([i+1 for i in range(labels.size(-1))])).unsqueeze(0)
    score_matrix = pos_matrix / sorted_ground_truth_ranks
    score_mask_matrix, _ = torch.sort(labels, dim=-1, descending=True)
    scores = score_matrix * score_mask_matrix
    score = scores.sum() / labels.sum()
    return score.item()

# # Sample usage
# y_true = torch.tensor(np.array([[1, 1, 0], [1, 0, 1], [0, 0, 1]]))
# y_score = torch.tensor(np.random.randn(3, 3))
# print(LRAP(y_score, y_true), LWLRAP(y_score, y_true))

In [13]:
def mixup_data(x, y, alpha=5):
    """
    Applies mixup to a sample
    Arguments:
        x {torch tensor} -- Input batch
        y {torch tensor} -- Labels
    Keyword Arguments:
        alpha {float} -- Parameter of the beta distribution (default: {0.4})
    Returns:
        torch tensor  -- Mixed input
        torch tensor  -- Labels of the original batch
        torch tensor  -- Labels of the shuffle batch
        float  -- Probability samples by the beta distribution
    """
    lam = np.random.beta(alpha, alpha) if alpha > 0 else 1
    index = torch.randperm(x.size()[0]).cuda()
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]

    return mixed_x, y_a, y_b, lam

# for step, (x, y_batch) in enumerate(train_loader):
    
#     if np.random.rand() < mixup_proba:
#         x, y_a, y_b, _ = mixup_data(x.cuda(), y_batch.cuda(), alpha=alpha)
#         y_batch = torch.clamp(y_a + y_b, 0, 1)

In [20]:
def validate(model, val_dataloader, criterion, history, n_iters, writer, fold):
    model.eval()
    costs = []
    rocs = []
    metrics = []
    y_trues = []
    y_preds = []
    y_probs = []
    with torch.no_grad():
        for idx, batch in enumerate(val_dataloader):
            X, y = batch
            X, y = X.cuda(), y.cuda()
            y_output = model(X)    
            loss = criterion(y_output, y)
            costs.append(loss.item())
            y_true, y_prob= y.cpu(), y_output['clipwise_output'].detach().cpu()
            y_pred = (y_prob+0.5).int()
            y_trues.append(y_true.numpy())
            y_probs.append(y_prob.numpy())
            y_preds.append(y_pred.numpy())
            metrics.append(LWLRAP(y_prob, y.cpu()))
    
    mean_rocs = sklearn.metrics.roc_auc_score(np.concatenate(y_trues), np.concatenate(y_probs))
    mean_costs = np.mean(costs)
    mean_metrics = np.mean(metrics)
    writer.add_scalar('fold_{}/validate_roc'.format(fold), mean_rocs, n_iters)
    writer.add_scalar('fold_{}/validate_loss'.format(fold), mean_costs, n_iters)
    writer.add_scalar('fold_{}/validate_LWLRAP'.format(fold), mean_metrics, n_iters)
    if mean_metrics > history['best_metrics'][fold]: 
        history['best_roc'][fold] = mean_rocs
        history['best_metrics'][fold] = mean_metrics
        torch.save(model.state_dict(), history['best_model_path'][fold])
    return mean_costs, mean_rocs, mean_metrics


def train(model, train_dataloader, val_dataloader, criterion, optimizer, epoch, history, validate_points, scheduler, writer, fold, step=True):
    model.train()
    costs = []
    metrics = []
    y_trues = []
    y_preds = []
    y_probs = []
    val_loss, val_roc = 0, 0
    optimizer.zero_grad()
    with tqdm(total=len(train_dataloader.dataset), desc='Epoch{}'.format(epoch)) as pbar:
        for idx, batch in enumerate(train_dataloader):
            X, y = batch
            X, y = X.cuda(), y.cuda()
            if np.random.rand() < Config.mixup_proba:
                X, y_a, y_b, _ = mixup_data(X, y, alpha=Config.mixup_alpha)
                y = torch.clamp(y_a + y_b, 0, 1)
            y_output = model(X)    
            loss = criterion(y_output, y)
            optimizer.zero_grad()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 3)
            optimizer.step()
            if step:
                scheduler.step()
            costs.append(loss.item())
            y_true, y_prob= y.cpu(), y_output['clipwise_output'].detach().cpu()
            y_pred = (y_prob+0.5).int()
            y_trues.append(y_true.numpy())
            y_probs.append(y_prob.numpy())
            y_preds.append(y_pred.numpy())
            metrics.append(LWLRAP(y_prob, y_true))
#             rocs.append(sklearn.metrics.roc_auc_score(y.cpu(), y_prob))
#                 rocs.append((y_pred == y.cpu()).float().mean().item())
            pbar.update(y.size(0))
            n_iters = idx + len(train_dataloader) * (epoch-1)
            if idx in validate_points:
                val_loss, val_roc, val_metrics = validate(model, val_dataloader, criterion, history, n_iters, writer, fold)
                model.train()
            writer.add_scalar('fold_{}/train_loss'.format(fold), costs[-1], n_iters)
            writer.add_scalar('fold_{}/train_LWLRAP'.format(fold), metrics[-1], n_iters)
            writer.add_scalar('fold_{}/learning_rate'.format(fold), scheduler.get_last_lr()[0], n_iters)
            pbar.set_postfix_str('loss:{:.3f}, val-loss:{:.3f}, val-roc:{:.4f}'.format(np.mean(costs[-10:]),  val_loss, val_roc))
            torch.cuda.empty_cache()
        writer.add_scalar('fold_{}/train_roc'.format(fold), sklearn.metrics.roc_auc_score(np.concatenate(y_trues), np.concatenate(y_probs)), n_iters)
        
            

In [21]:
class PANNsLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.bce = nn.BCELoss()

    def forward(self, input, target):
        input_ = input["clipwise_output"]
        input_ = torch.clamp(input_, 0, 1)
        target = target.float()
        return self.bce(input_, target)

In [ ]:
# pos_weights = torch.ones(Config.num_class)
# counts = data_tp_df['species_id'].value_counts()
# for i in range(Config.num_class):
#     pos_weights[i] = (sum(counts)-counts[i])/counts[i]
# loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights).cuda()

# def criterion(y_pred, y_target):
#     loss = loss_function(y_pred, y_target.float())
#     return loss
criterion = PANNsLoss().cuda()


time_stamp = '{0:%m_%d_%H_%M}'.format(datetime.datetime.now())
history = {
    'config': Config,
    'best_roc': [0]*len(data_folds),
    'best_metrics': [0]*len(data_folds), 
    'best_model_path': [os.path.join(model_path, '{}_{}_fold_{}.pth'.format(Config.model_name, time_stamp, i)) for i in range(len(data_folds))]
}
writer = SummaryWriter(log_dir=os.path.join(tensorboard_path, '{}_{}'.format(Config.model_name, time_stamp)))
for idx, (train_dataloader, val_dataloader) in enumerate(data_folds):
    validate_points = list(np.linspace(0, len(train_dataloader)-1, 2).astype(int))[1:]
    model = Net().cuda()
#     model = nn.DataParallel(model, device_ids=[0, 1])
    epochs = 30
    warmup_prob = 0.1
    optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-3, epochs=epochs, steps_per_epoch=len(train_dataloader), pct_start=warmup_prob, div_factor=25, anneal_strategy='cos', cycle_momentum=True)
#     scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(warmup_prob*len(train_dataloader)*epochs), num_training_steps=len(train_dataloader)*epochs)
    for epoch in range(1, epochs+1):
        train(model, train_dataloader, val_dataloader, criterion, optimizer, epoch, history, validate_points, scheduler,  writer, fold=idx, step=True)
#         scheduler.step()
        gc.collect()
#     torch.save(model.state_dict(), history['best_model_path'][idx])
    del model
    gc.collect()
    torch.cuda.empty_cache()
    
    
with open(os.path.join(model_path, '{}_{}_history.pkl'.format(Config.model_name, time_stamp)), 'wb') as f:
    pickle.dump(history, f)

Epoch1: 100%|██████████| 972/972 [00:41<00:00, 51.36it/s, loss:0.381, val-loss:0.255, val-roc:0.4978]

0.1581940020202078 0
save_model


## Predict Testset

In [14]:
# for file in os.listdir(model_path):
#     if file.endswith('.pkl'):
#         print(file)

resnest_augment_RandomCrop_mixup_256_600_02_08_03_40_history.pkl


In [17]:
# with open(os.path.join(model_path, 'resnest_augment_RandomCrop_mixup_256_600_02_08_03_40_history.pkl'), 'rb') as f:
#     history = pickle.load(f)
model = Net().cuda()

In [19]:
history

{'config': __main__.Config,
 'best_roc': [0.982298636332259,
  0.9831462753720066,
  0.9764797753874944,
  0.9701418814880866,
  0.9774833974142592],
 'best_metrics': [0.8677614182692308,
  0.8792319723942208,
  0.8869343361659356,
  0.8781604891134085,
  0.8872028704573935],
 'best_model_path': ['/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_0.pth',
  '/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_1.pth',
  '/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_2.pth',
  '/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_3.pth',
  '/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_4.pth']}

In [18]:
folds = []
for path in history['best_model_path']:
    model.load_state_dict(torch.load(path, map_location= torch.device('cpu')), strict=True)
    model.eval()
    preds = []
    with torch.no_grad():
        for batch in tqdm(test_dataloader):
            a, b, c, d, e = batch.size()
            X = batch.view(a*b, c, d, e).cuda()
            output = model(X, attn_mask=False)
            pred1 = output['clipwise_output'].view(a, b, 24).max(dim=1)[0].cpu().detach().numpy()

            pred2 = output['framewise_output'].view(a, b, -1, 24).max(dim=2)[0].max(dim=1)[0].cpu().detach().numpy()
            preds.append((pred1+pred2)/2)
    folds.append(np.concatenate(preds, axis=0))

FileNotFoundError: [Errno 2] No such file or directory: '/dev/shm/model_save/resnet34d_augment_RandomCrop_mixup_attention_02_14_04_25_fold_0.pth'

In [ ]:
sub = pd.DataFrame(columns=['recording_id','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23'], dtype=np.float32)
sub['recording_id'] = [file.split('.')[0] for file in test_files]
sub.iloc[:, 1:] = sum(folds)/len(folds)

In [ ]:
time_stamp = '{0:%m_%d_%H_%M}'.format(datetime.datetime.now())
sub.to_csv(os.path.join(res_path, 'submission_{}_{}_{:.4f}.csv'.format(Config.model_name, time_stamp, np.mean(history['best_metrics']))), index=None)

In [19]:
np.mean(history['best_acc'])

0.9317194581031799

In [20]:
np.mean(history['best_metrics'])

0.8855923084819054